# Attitude Dynamics and Control of a Nano-Satellite Orbiting Mars

**Mission Brief**

A nano-satellite in circular Low Mars Orbit (LMO) must autonomously execute attitude control over a 6500 s mission. A second spacecraft (the mothercraft) operates in circular Geosynchronous Mars Orbit (GMO). The nano-satellite must switch between three operational modes:

• Science Mode — point a surface sensor toward Mars (nadir pointing).  
• Power Mode — orient solar panels toward the Sun.  
• Communication Mode — point the antenna toward the mothercraft.

Both spacecraft are modeled using two-body circular orbit dynamics.

The inertial frame is defined as  
$\mathcal{N} = \{\hat{n}_1, \hat{n}_2, \hat{n}_3\}$.

The LMO orbit is described using the passive 3-1-3 Euler sequence  
$(\Omega, i, \theta)$, where  
$\Omega$ = right ascension of ascending node,  
$i$ = inclination,  
$\theta$ = true latitude.

The Hill frame associated with each circular orbit is  

$\mathcal{H} = \{\hat{r}, \hat{\theta}, \hat{h}\}$,

where  
$\hat{r}$ is radial,  
$\hat{\theta}$ is along-track,  
$\hat{h}$ is orbit normal.

For circular two-body motion, the orbital angular rate is constant and given by

$$
\dot{\theta} = \sqrt{\frac{\mu}{r^3}}
$$

where $\mu$ is the Mars gravitational parameter.

Mars has a rotation period of 1 day and 37 minutes. The GMO orbit therefore has the same period and zero inclination (equatorial orbit). Both LMO and GMO angular rates are constant.

The spacecraft body frame is defined as  

$\mathcal{B} = \{\hat{b}_1, \hat{b}_2, \hat{b}_3\}$,

with hardware alignment:
• $+\hat{b}_1$ — science sensor  
• $-\hat{b}_1$ — communication antenna  
• $+\hat{b}_3$ — solar panel normal  

Attitude control is implemented using a PD tracking law:

$$
\mathbf{u}_B = -K\,\boldsymbol{\sigma}_{B/R} - P\,\boldsymbol{\omega}_{B/R}
$$

where $\boldsymbol{\sigma}_{B/R}$ are Modified Rodrigues Parameters and  
$\boldsymbol{\omega}_{B/R}$ is the relative angular velocity.

Spacecraft rotational dynamics follow Euler’s rotational equation of motion:

$$
[I]\dot{\boldsymbol{\omega}} =
-\tilde{\boldsymbol{\omega}}[I]\boldsymbol{\omega} + \mathbf{u}
$$

The objective of this mission simulation is to:

• Generate time-varying orbital reference frames for LMO and GMO.  
• Construct reference attitudes for each operational mode.  
• Compute attitude tracking errors.  
• Integrate full nonlinear attitude dynamics using RK4.  
• Autonomously switch control modes based on eclipse and line-of-sight geometry.  
• Export validated numerical outputs in reproducible text format.

This notebook develops the required kinematics, reference generation, control law implementation, and nonlinear simulation required to complete the mission.
simulation.

In [1]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

from pathlib import Path

import sys
sys.path.insert(0, r"../../")
import AttitudeKinematicsLib as ak

In [2]:
print("Contents of AttitudeKinematicsLib:")
for name in sorted(dir(ak)):
    if not name.startswith("_"):
        print(name)

Contents of AttitudeKinematicsLib:
BInvmat_CRP
BInvmat_EP
BInvmat_Euler
BInvmat_MRP
BInvmat_PRV
Bmat_CRP
Bmat_EP
Bmat_Euler
Bmat_MRP
Bmat_PRV
CRP
CRP_to_DCM
DCM_to_CRP
DCM_to_EP
DCM_to_Euler
DCM_to_MRP
DCM_to_PRV
DCM_utils
EP_to_DCM
EulerAngles
EulerRodriguesParameters
Euler_to_DCM
MRP
MRP_shadow
MRP_to_DCM
PRV
PRV_to_DCM
integrate_quaternion
normalize_quat
np
quat_derivative
quat_diff
quat_inv
quat_mult
rotation_matrix_x
rotation_matrix_y
rotation_matrix_z
skew_symmetric
solve_ivp
validate_DCM
validate_vec3
validate_vec4


# Module 1 - Practice Assignment

In [3]:
OUTPUT_DIR = Path("outputs")
OUTPUT_DIR.mkdir(exist_ok=True)

In [4]:
def export_txt(filename_stem: str, data, precision: int = 4) -> None:
    """
    Export mission result data to the outputs directory in Coursera format.

    Parameters
    ----------
    filename_stem : str
        Name of the output file without extension.
    data : scalar | array-like | str
        Numerical data to export. Scalars are written as a single value.
        1D vectors are written as "a b c".
        Strings are written as-is.
    precision : int, optional
        Decimal precision for numerical formatting (default: 12).
    """
    file_path = OUTPUT_DIR / f"{filename_stem}.txt"

    # Case 1: data is a string
    if isinstance(data, str):
        text = data.strip()
        if text.startswith("[") and text.endswith("]"):
            text = text[1:-1].strip()
        text = text.replace(",", " ")
        text = " ".join(text.split())

    # Case 2: numerical data
    else:
        arr = np.asarray(data)

        # Flatten ANY numeric input to 1D row-major
        flat = arr.astype(float).ravel(order="C")

        text = " ".join(f"{val:.{precision}f}" for val in flat)

    file_path.write_text(text, encoding="utf-8")
    print(f"[Saved] {file_path} -> {text}")


In [5]:
part1 = " ".join(str(x) for x in range(1, 10, 2))
print(part1)

export_txt(filename_stem="module1_part1", data=part1)

1 3 5 7 9
[Saved] outputs/module1_part1.txt -> 1 3 5 7 9


In [6]:
A = [[1, 2], 
     [9, 10]]

part2 = " ".join(str(A[i][j]) for i in range(len(A)) for j in range(len(A[0])))
print(part2)

export_txt(filename_stem="module1_part2", data=part2)

1 2 9 10
[Saved] outputs/module1_part2.txt -> 1 2 9 10


In [7]:
part3 = f"{np.pi:.5f}"
print(part3)

export_txt(filename_stem="module1_part3", data=part3)

3.14159
[Saved] outputs/module1_part3.txt -> 3.14159


# Module 2 - Orbits

In [8]:

R_MARS_km = 3396.19
mu_MARS  = 42828.3

h_km  = 400.0

# LMO
r_LMO_km  = R_MARS_km + h_km
theta_dot_LMO = np.sqrt(mu_MARS/r_LMO_km**3)     # rad/s
Omega_LMO_deg, inc_LMO_deg, theta0_LMO_deg = 20.0, 30.0, 60.0
LMO_angles_deg = np.array([Omega_LMO_deg, inc_LMO_deg, theta0_LMO_deg])
LMO_angles_rad = np.array([np.deg2rad(i) for i in LMO_angles_deg])

# GMO
r_GMO_km  = 20424.2
theta_dot_GMO = np.sqrt(mu_MARS/r_GMO_km**3)     # rad/s
Omega_GMO_deg, inc_GMO_deg, theta0_GMO_deg = 0.0,  0.0,  250.0
GMO_angles_deg = np.array([Omega_GMO_deg, inc_GMO_deg, theta0_GMO_deg])
GMO_angles_rad = np.array([np.deg2rad(i) for i in GMO_angles_deg])

# Initial attitude and Body rate
sigma_BN = np.array([0.3, -0.4, 0.5])
omega_BN_B = np.array([np.deg2rad(i) for i in [1.00, 1.75, -2.20]]).T

# Satellite inertia tensor
I = np.array([[10, 0,   0],
              [ 0, 5,   0],
              [ 0, 0, 7.5]])


## Task 1 - Hill-to-Inertial State Transformation for Circular Orbits

<p align="center">
  <img src="images/figure_1.png" width="1000"/>
</p>

The spacecraft is assumed to move in a circular orbit about Mars. Two reference frames are defined:

- The inertial frame $\mathcal{N} = \{\hat{n}_1,\hat{n}_2,\hat{n}_3\}$.
- The Hill (orbit) frame $\mathcal{H} = \{\hat{i}_r,\hat{i}_\theta,\hat{i}_h\}$, where  
  $\hat{i}_r$ is radial,  
  $\hat{i}_\theta$ is along-track, and  
  $\hat{i}_h$ is orbit normal.

For a circular orbit, the position vector is aligned with the radial direction:

$$
\mathbf{r} = r\,\hat{i}_r
\qquad \Rightarrow \qquad
{}^H\mathbf{r} =
\begin{bmatrix}
r \\
0 \\
0
\end{bmatrix}.
$$

To obtain the inertial velocity, we apply the transport theorem:

$$
\left(\frac{d\mathbf{r}}{dt}\right)_{\mathcal{N}}
=
\left(\frac{d\mathbf{r}}{dt}\right)_{\mathcal{H}}
+
\boldsymbol{\omega}_{\mathcal{H}/\mathcal{N}} \times \mathbf{r}.
$$

Since $\mathbf{r}=r\hat{i}_r$,

$$
\left(\frac{d\mathbf{r}}{dt}\right)_{\mathcal{H}}
=
\dot{r}\,\hat{i}_r
+
r\left(\frac{d\hat{i}_r}{dt}\right)_{\mathcal{H}}.
$$

Because $\hat{i}_r$ is fixed in the Hill frame,

$$
\left(\frac{d\hat{i}_r}{dt}\right)_{\mathcal{H}} = \mathbf{0},
$$

so

$$
\left(\frac{d\mathbf{r}}{dt}\right)_{\mathcal{H}} = \dot{r}\,\hat{i}_r.
$$

For a circular orbit, $\dot{r}=0$, and the Hill frame rotates about $\hat{i}_h$ with constant rate

$$
\boldsymbol{\omega}_{\mathcal{H}/\mathcal{N}} = \dot{\theta}\,\hat{i}_h.
$$

Substituting into the transport theorem gives

$$
\dot{\mathbf{r}}
=
r\dot{\theta}\,
(\hat{i}_h \times \hat{i}_r).
$$

Using the Hill-frame right-hand rule $\hat{i}_h \times \hat{i}_r = \hat{i}_\theta$,

$$
\dot{\mathbf{r}} = r\dot{\theta}\,\hat{i}_\theta
\qquad \Rightarrow \qquad
{}^H\dot{\mathbf{r}} =
\begin{bmatrix}
0 \\
r\dot{\theta} \\
0
\end{bmatrix}.
$$

Thus, the orbital kinematics are fully defined in the Hill frame.

The orientation of the Hill frame relative to inertial is defined using the passive 3-1-3 Euler sequence $(\Omega, i, \theta)$:

$$
[HN] = R_3(\theta)\,R_1(i)\,R_3(\Omega).
$$

Because this DCM maps inertial components into Hill components, inertial vectors are obtained using its transpose:

$$
{}^N\mathbf{r} = [HN]^T\,{}^H\mathbf{r},
\qquad
{}^N\dot{\mathbf{r}} = [HN]^T\,{}^H\dot{\mathbf{r}}.
$$

A useful geometric check of correctness is

$$
r_{N,3} = r \sin i \sin\theta,
$$

which must hold for a properly constructed DCM.

This task therefore requires us to compute the time-updated true latitude $\theta(t)$, construct the passive 3-1-3 DCM defining the Hill frame orientation, transform the canonical Hill-frame position and velocity vectors into inertial coordinates, and export the resulting inertial components in the required format.


In [9]:
def circular_orbit_rv_N(angles_deg, d_angles_deg, r, t_s):
    """
    Compute inertial position and velocity for a circular orbit
    using time-updated 3-1-3 Euler angles.

    Parameters
    ----------
    angles_deg : array-like, shape (3,)
        Initial 3-1-3 Euler angles [Omega, inc, theta] in degrees.
    d_angles_deg : array-like, shape (3,)
        Time derivatives of Euler angles [Omega_dot, inc_dot, theta_dot] in deg/s.
        For circular two-body motion: typically [0, 0, theta_dot_deg].
    r : float
        Orbit radius (consistent units).
    t_s : float
        Time in seconds.

    Returns
    -------
    r_N : ndarray (3,)
        Inertial position vector.
    v_N : ndarray (3,)
        Inertial velocity vector.
    """

    angles_deg = np.asarray(angles_deg, dtype=float)
    d_angles_deg = np.asarray(d_angles_deg, dtype=float)

    # Update Euler angles linearly in time
    angles_t = angles_deg + d_angles_deg * t_s

    # Build passive Hill DCM [HN]
    HN = ak.Euler_to_DCM(
        angles=angles_t,
        sequence="313",
        transformation_type="passive"
    )

    NH = HN.T

    # Hill-frame canonical circular-orbit vectors
    theta_dot_rad = np.deg2rad(d_angles_deg[2])  # convert deg/s -> rad/s
    r_H = np.array([r, 0.0, 0.0])
    v_H = np.array([0.0, r * theta_dot_rad, 0.0])

    # Transform to inertial
    r_N = NH @ r_H
    v_N = NH @ v_H

    return r_N, v_N


In [10]:
t_LMO = 450.0   # s
t_GMO = 1150.0  # s

# Convert angular rates to deg/s for the Euler-angle update
theta_dot_LMO_deg = np.rad2deg(theta_dot_LMO)
theta_dot_GMO_deg = np.rad2deg(theta_dot_GMO)

# Initial Euler angles (Omega, inc, theta0)
angles_LMO = np.array([Omega_LMO_deg, inc_LMO_deg, theta0_LMO_deg])
angles_GMO = np.array([Omega_GMO_deg, inc_GMO_deg, theta0_GMO_deg])

# Time derivatives of Euler angles for circular two-body
# Omega_dot = 0, inc_dot = 0, theta_dot = constant
d_angles_LMO = np.array([0.0, 0.0, theta_dot_LMO_deg])
d_angles_GMO = np.array([0.0, 0.0, theta_dot_GMO_deg])

# Task 1 Parts 1 & 2 (LMO at 450 s)
rN_LMO, vN_LMO = circular_orbit_rv_N(angles_LMO, d_angles_LMO, r_LMO_km,t_LMO)
export_txt("task1_part1", rN_LMO)
export_txt("task1_part2", vN_LMO)

# Task 1 Parts 3 & 4 (GMO at 1150 s)
rN_GMO, vN_GMO = circular_orbit_rv_N(angles_GMO, d_angles_GMO, r_GMO_km, t_GMO)
export_txt("task1_part3", rN_GMO)
export_txt("task1_part4", vN_GMO)

[Saved] outputs/task1_part1.txt -> -669.2851 3227.4983 1883.1811
[Saved] outputs/task1_part2.txt -> -3.2560 -0.7978 0.2101
[Saved] outputs/task1_part3.txt -> -5399.1504 -19697.6425 0.0000
[Saved] outputs/task1_part4.txt -> 1.3966 -0.3828 0.0000


## Task 2 - Orbit Frame Orientation

In [11]:
def HN_LMO(t_s: float) -> np.ndarray:
    """
    Return the passive Hill DCM [HN(t)] for the LMO orbit at time t [s].
    Uses the 3-1-3 Euler sequence (Omega, i, theta(t)).
    """
    theta_t_deg = theta0_LMO_deg + np.rad2deg(theta_dot_LMO * t_s)

    # Optional: wrap for interpretability only (not required)
    # theta_t_deg %= 360.0

    HN = ak.Euler_to_DCM(
        angles=[Omega_LMO_deg, inc_LMO_deg, theta_t_deg], 
        sequence="313",
        transformation_type="passive"
    )
    return HN

t_test = 300.0  # [s]
HN_300 = HN_LMO(t_test)

print("HN(t=300s) =\n", HN_300)
export_txt("task2_part3", HN_300, precision=6)



HN(t=300s) =
 [[-0.0464774   0.87414792  0.48343072]
 [-0.98417245 -0.12292213  0.12765086]
 [ 0.17101007 -0.46984631  0.8660254 ]]
[Saved] outputs/task2_part3.txt -> -0.046477 0.874148 0.483431 -0.984172 -0.122922 0.127651 0.171010 -0.469846 0.866025


# Module 3 - Reference Frame Orientation

## Task 3 - Sun-Pointing Reference Frame Orientation

<p align="center">
  <img src="images/figure_2.png" width="800"/>
</p>

<p align="center">
  <img src="images/figure_3.png" width="800"/>
</p>

To orient the spacecraft solar panels toward the Sun, the reference frame $\mathcal{R}_s$ must be defined such that its third axis aligns with the Sun direction. In this scenario, the Sun direction is fixed along $\hat{n}_2$ in the inertial frame $\mathcal{N}$.

Thus we impose:

$$
\hat{r}_3 = \hat{n}_2
$$

Additionally, the problem specifies that the first axis must point in the $-\hat{n}_1$ direction:

$$
\hat{r}_1 = -\hat{n}_1
$$

To complete a right-handed orthonormal triad, the second axis is obtained from the cross product:

$$
\hat{r}_2 = \hat{r}_3 \times \hat{r}_1
$$

Substituting the definitions:

$$
\hat{r}_2 = \hat{n}_2 \times (-\hat{n}_1)
$$

Using the inertial right-hand rule $\hat{n}_1 \times \hat{n}_2 = \hat{n}_3$, we obtain:

$$
\hat{r}_2 = \hat{n}_3
$$

Therefore the Sun-pointing reference frame expressed in inertial components is:

$$
\hat{r}_1 = [-1, 0, 0], \quad
\hat{r}_2 = [0, 0, 1], \quad
\hat{r}_3 = [0, 1, 0]
$$

Stacking these basis vectors as rows yields the DCM:

$$
[R_s N] =
\begin{bmatrix}
-1 & 0 & 0 \\
0 & 0 & 1 \\
0 & 1 & 0
\end{bmatrix}
$$

Because each axis of $\mathcal{R}_s$ is aligned with fixed inertial directions, this reference frame does not rotate relative to $\mathcal{N}$. Hence,

$$
{}^N\boldsymbol{\omega}_{R_s/N} = \mathbf{0}
$$


In [12]:
def RsN(t_s: float) -> np.ndarray:
    """
    Sun-pointing reference frame DCM [RsN].

    Definition from brief:
    r3 = +n2 (Sun direction)
    r1 = -n1
    r2 = r3 x r1 = +n3

    This frame is constant in inertial for this scenario.
    """
    r1_N = np.array([-1.0, 0.0, 0.0])  # -n1
    r2_N = np.array([ 0.0, 0.0, 1.0])  # +n3
    r3_N = np.array([ 0.0, 1.0, 0.0])  # +n2

    # DCM rows are the Rs basis vectors expressed in N components
    RsN = np.vstack((r1_N, r2_N, r3_N))
    return RsN


def omega_RsN(t_s: float) -> np.ndarray:
    """
    Inertial angular velocity of Rs relative to N, expressed in N.

    Since Rs is fixed to inertial directions (n2 and -n1), it does not rotate.
    """
    return np.zeros(3)

t0 = 0.0
RsN_0 = RsN(t0)
omega_0 = omega_RsN(t0)

print("RsN(t=0) =\n", RsN_0)
print("N omega_Rs/N(t=0) =", omega_0)

export_txt("task3_part1", RsN_0, precision=6)   # matrix -> row-major single line
export_txt("task3_part2", omega_0, precision=12) # vector -> "a b c"

RsN(t=0) =
 [[-1.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]]
N omega_Rs/N(t=0) = [0. 0. 0.]
[Saved] outputs/task3_part1.txt -> -1.000000 0.000000 0.000000 0.000000 0.000000 1.000000 0.000000 1.000000 0.000000
[Saved] outputs/task3_part2.txt -> 0.000000000000 0.000000000000 0.000000000000


## Task 4 - Nadir-Pointing Reference Frame Orientation

Task 4 is not about computing another random DCM. It is about understanding how reference frames are built from geometry.

The goal is to define a *nadir-pointing* reference frame $\mathcal{R}_n$.

What does "nadir-pointing" actually mean?

It means the spacecraft’s sensor axis must point toward the center of Mars. In orbital geometry, the vector pointing from the spacecraft to Mars is the negative radial direction:

$$
\hat{r}_1 = -\hat{i}_r
$$

The problem further specifies that the second axis of the reference frame must point in the velocity direction:

$$
\hat{r}_2 = \hat{i}_\theta
$$

Now pause.

We are not inventing new geometry. We are reusing the Hill frame:

- $\hat{i}_r$ = radial direction  
- $\hat{i}_\theta$ = along-track (velocity direction)  
- $\hat{i}_h$ = orbit normal  

So the nadir frame is simply a reoriented Hill frame.

To complete a right-handed orthonormal triad, the third axis must satisfy:

$$
\hat{r}_3 = \hat{r}_1 \times \hat{r}_2
$$

Substitute the definitions:

$$
\hat{r}_3 = (-\hat{i}_r) \times (\hat{i}_\theta)
$$

Since the Hill triad obeys

$$
\hat{i}_r \times \hat{i}_\theta = \hat{i}_h
$$

we obtain

$$
\hat{r}_3 = -\hat{i}_h
$$

So geometrically, the nadir frame is:

$$
\hat{r}_1 = -\hat{i}_r, \quad
\hat{r}_2 = \hat{i}_\theta, \quad
\hat{r}_3 = -\hat{i}_h
$$

Now the key insight:

We already know how the Hill frame relates to inertial:

$$
\{i\} = [H_s N]\{n\}
$$

So instead of rebuilding everything from scratch, we express the nadir axes in terms of Hill axes, and then chain the transformations.

If we define a constant matrix that maps Hill to nadir,

$$
[H R_n] =
\begin{bmatrix}
-1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & -1
\end{bmatrix}
$$

then the full inertial-to-nadir mapping becomes:

$$
[R_n N] = [H R_n]^T [H_s N]
$$

What are we really doing?

We are composing coordinate transformations:

1. Inertial → Hill  
2. Hill → Nadir  

That’s all.

No new dynamics.
No new physics.
Just a different choice of orientation built from the orbit geometry.

Finally, what about angular velocity?

The nadir frame is rigidly attached to the Hill frame (only constant sign flips). Since constant rotations do not change angular velocity, we have:

$$
{}^N\boldsymbol{\omega}_{R_n/N}
=
{}^N\boldsymbol{\omega}_{H/N}
=
\dot{\theta} \hat{i}_h
$$

So Task 4 is simply this:

- Reinterpret the Hill frame to satisfy nadir constraints.
- Compose DCMs correctly.
- Recognize that angular velocity follows from orbit motion, not from arbitrary matrix manipulation.


In [13]:

def RnN(t_s: float) -> np.ndarray:
    """
    Nadir-pointing reference frame DCM [RnN(t)].

    Definitions:
      r1 = -i_r   (points to Mars / nadir)
      r2 = +i_theta (velocity direction)
      r3 = r1 x r2  (right-handed completion) = -i_h
    """
    HN_t = HN_LMO(t_s)

    # Hill unit vectors expressed in N are the rows of [HN]
    i_r_N     = HN_t[0, :]
    i_theta_N = HN_t[1, :]
    i_h_N     = HN_t[2, :]

    r1_N = -i_r_N
    r2_N =  i_theta_N
    r3_N = np.cross(r1_N, r2_N)  # should equal -i_h_N (up to numerical noise)

    # Normalize to be safe
    r1_N = r1_N / np.linalg.norm(r1_N)
    r2_N = r2_N / np.linalg.norm(r2_N)
    r3_N = r3_N / np.linalg.norm(r3_N)

    # Stack as rows: {r} = [RnN]{n}
    RnN_t = np.vstack((r1_N, r2_N, r3_N))
    return RnN_t


def omega_RnN(t_s: float) -> np.ndarray:
    """
    Inertial angular velocity of Rn relative to N, expressed in N.

    Rn differs from the Hill frame by a constant axis flip, so it has the same
    angular velocity as the Hill frame:
      N w_Rn/N = N w_H/N = theta_dot * i_h (expressed in N).
    """
    HN_t = HN_LMO(t_s)
    i_h_N = HN_t[2, :]  # Hill normal expressed in N
    return theta_dot_LMO * i_h_N

t_eval = 330.0

RnN_330 = RnN(t_eval)
omega_330 = omega_RnN(t_eval)

print("RnN(330s) =\n", RnN_330)
print("N omega_Rn/N(330s) =", omega_330)

export_txt("task4_part1", RnN_330, precision=6)
export_txt("task4_part2", omega_330, precision=12)

RnN(330s) =
 [[ 0.07258174 -0.87057754 -0.48664837]
 [-0.98259221 -0.1460794   0.11477527]
 [-0.17101007  0.46984631 -0.8660254 ]]
N omega_Rn/N(330s) = [ 0.00015131 -0.00041572  0.00076626]
[Saved] outputs/task4_part1.txt -> 0.072582 -0.870578 -0.486648 -0.982592 -0.146079 0.114775 -0.171010 0.469846 -0.866025
[Saved] outputs/task4_part2.txt -> 0.000151309151 -0.000415718477 0.000766256442


## Task 5 - GMO-pointing reference frame orientation

Task 5 introduces the communication mode reference frame $\mathcal{R}_c$.

Let us think about what "communication mode" actually means.

The spacecraft antenna must point toward the geosynchronous Mars orbiter (GMO). Therefore, the primary axis of the reference frame must align with the line-of-sight vector from the LMO spacecraft to the GMO spacecraft.

So step one is not about matrices. It is about geometry.

At any time $t$:

- The LMO position in inertial space is ${}^N\mathbf{r}_{LMO}(t)$.
- The GMO position in inertial space is ${}^N\mathbf{r}_{GMO}(t)$.

The vector pointing from LMO to GMO is:

$$
{}^N\mathbf{\rho}(t) = {}^N\mathbf{r}_{GMO}(t) - {}^N\mathbf{r}_{LMO}(t)
$$

This is the communication direction.

To turn this into a reference axis, we normalize it:

$$
\hat{r}_{1,c}(t) = \frac{{}^N\mathbf{\rho}(t)}{\|{}^N\mathbf{\rho}(t)\|}
$$

That defines the first axis of $\mathcal{R}_c$.

Now pause.

One vector does not define a frame. We need three orthonormal axes.

To build a stable right-handed frame, we choose a secondary direction that gives geometric meaning. A natural choice is the orbit normal $\hat{i}_h$ of the LMO orbit, since it is perpendicular to the orbital plane and always well-defined.

Using that, define the second axis as:

$$
\hat{r}_{2,c}(t) = \frac{\hat{i}_h(t) \times \hat{r}_{1,c}(t)}{\|\hat{i}_h(t) \times \hat{r}_{1,c}(t)\|}
$$

This ensures $\hat{r}_{2,c}$ is perpendicular to both the orbit normal and the communication line.

Then complete the triad:

$$
\hat{r}_{3,c}(t) = \hat{r}_{1,c}(t) \times \hat{r}_{2,c}(t)
$$

Now we have a full orthonormal basis.

Stack these axes as rows to form the DCM:

$$
[R_c N](t) =
\begin{bmatrix}
\hat{r}_{1,c}^T \\
\hat{r}_{2,c}^T \\
\hat{r}_{3,c}^T
\end{bmatrix}
$$

That completes the orientation definition.

Now comes the deeper question:

What is the angular velocity of this frame?

Unlike the Sun frame (constant) or nadir frame (orbit-locked), the communication frame depends on the *relative motion of two spacecraft*. Its orientation changes in a nonlinear way because the line-of-sight vector changes direction as both orbits evolve.

There is no simple closed-form expression for ${}^N\boldsymbol{\omega}_{R_c/N}$.

But there is a general identity for any rotating DCM:

$$
\tilde{\boldsymbol{\omega}}_{R_c/N}
=
\dot{R}_{cN} R_{cN}^T
$$

So if we compute the time derivative numerically,

$$
\dot{R}_{cN}(t) \approx \frac{R_{cN}(t+\Delta t) - R_{cN}(t-\Delta t)}{2\Delta t}
$$

then extract the skew-symmetric part, we recover the angular velocity vector.

This is not "cheating".

It is using a fundamental kinematic identity:

The time rate of change of a DCM encodes angular velocity.

So Task 5 is conceptually simple:

1. Compute the inertial positions of LMO and GMO.
2. Build the line-of-sight vector.
3. Construct a right-handed orthonormal triad.
4. Stack into a DCM.
5. Use numerical differentiation of the DCM to recover angular velocity.

No magic.
Just geometry and kinematics.


In [14]:
def RcN(t_s: float) -> np.ndarray:
    """
    Communication-mode reference frame DCM [RcN](t).

    Reference construction (matches capstone solution style):
      Delta_r = r_GMO - r_LMO
      r1 = -Delta_r / ||Delta_r||                      (point from GMO to LMO, i.e. antenna points to GMO)
      r2 = (Delta_r x n3) / ||Delta_r x n3||           (use inertial n3 as the "up" reference)
      r3 = r1 x r2                                     (complete right-handed triad)

    Returns [RcN] with ROWS equal to r1^T, r2^T, r3^T (i.e. {r_c} = [RcN]{n}).
    """
    rN_LMO, _ = circular_orbit_rv_N(LMO_angles_deg, d_angles_LMO, r_LMO_km, t_s)
    rN_GMO, _ = circular_orbit_rv_N(GMO_angles_deg, d_angles_GMO, r_GMO_km, t_s)

    Delta_r = rN_GMO - rN_LMO
    r1_N = -Delta_r / np.linalg.norm(Delta_r)

    n3_N = np.array([0.0, 0.0, 1.0])
    r2_N = np.cross(Delta_r, n3_N)
    r2_N = r2_N / np.linalg.norm(r2_N)

    r3_N = np.cross(r1_N, r2_N)
    r3_N = r3_N / np.linalg.norm(r3_N)

    RcN = np.array([r1_N, r2_N, r3_N])
    return RcN

def omega_RcN(t_s: float, dt: float = 1e-3) -> np.ndarray:
    """
    Compute N w_Rc/N using numerical differentiation of [RcN].

    Uses central difference:
      Rdot ≈ (R(t+dt) - R(t-dt)) / (2 dt)

    Then:
      [w~] = Rdot * R^T   (skew-symmetric)
    Extract w from the skew matrix.
    """
    R = RcN(t_s)
    R_forward = RcN(t_s + dt)

    # Newton forward difference
    Rdot = (R_forward - R) / dt

    # Skew symmetric matrix
    Wtilde = np.dot(-R.T, Rdot)

    # Force exact skew-symmetry (kills numerical drift)
    Wtilde = 0.5 * (Wtilde - Wtilde.T)

    # Aguklar vel
    w1 = Wtilde[2, 1]
    w2 = Wtilde[0, 2]
    w3 = Wtilde[1, 0]
    omega_RcN = np.array([w1, w2, w3])
    return omega_RcN


t_eval = 330.0

RcN_330 = RcN(t_eval)
omega_330 = omega_RcN(t_eval, dt=1e-3)

print("RcN(330s) =\n", RcN_330)
print("omega_Rc/N(330s) =", omega_330)

export_txt("task5_part1", RcN_330, precision=6)     # matrix, row-major single line
export_txt("task5_part2", omega_330, precision=12)  # vector "a b c"


RcN(330s) =
 [[ 0.26547539  0.96092816  0.07835742]
 [-0.96389181  0.26629415  0.        ]
 [-0.02086612 -0.07552807  0.99692533]]
omega_Rc/N(330s) = [ 1.97828948e-05 -5.46541344e-06  1.91300006e-04]
[Saved] outputs/task5_part1.txt -> 0.265475 0.960928 0.078357 -0.963892 0.266294 0.000000 -0.020866 -0.075528 0.996925
[Saved] outputs/task5_part2.txt -> 0.000019782895 -0.000005465413 0.000191300006


# Module 4 - Attitude Evaluation and Simulator

## Task 6 - Attitude Error Evaluation

At this stage, all reference frames have already been defined:

- Sun-pointing frame R_s
- Nadir-pointing frame R_n
- Communication-pointing frame R_c

We also know the spacecraft body frame B at the initial time t0.

Task 6 is not about building new frames. It is about comparing frames.

The question being asked is:

"How far is the spacecraft body B from the desired reference frame R at time t0?"

This difference is expressed using:

    sigma_B/R   (MRP attitude error)
    B omega_B/R (relative angular velocity expressed in B)


**<ins>Attitude Error sigma_B/R</ins>**

We first compute the relative DCM:

    [BR] = [BN] [RN]^T

Why this works:

- [BN] maps inertial -> body
- [RN] maps inertial -> reference
- Therefore [RN]^T maps reference -> inertial
- So multiplying gives reference -> body

Once we have [BR], we convert it to MRPs:

    sigma_B/R = DCM_to_MRP([BR])

This gives the attitude error between spacecraft and reference.

If sigma = 0, the spacecraft is perfectly aligned.



**<ins>Angular Velocity Error B omega_B/R</ins>**

The relative angular velocity is:

    omega_B/R = omega_B/N - omega_R/N

But both must be expressed in the SAME frame.

The assignment asks for B omega_B/R, meaning:

    B omega_B/R = B omega_B/N - B omega_R/N

If angular velocities are known in inertial form:

    N omega_B/R = N omega_B/N - N omega_R/N

Then convert to body frame:

    B omega_B/R = [BN] (N omega_B/R)

At the initial time t0, if the spacecraft is assumed inertially fixed:

    omega_B/N = 0

Then:

    omega_B/R = - omega_R/N

So the reference frame motion entirely drives the relative rate.

So Task 6 simply is:

For each reference frame (Sun, Nadir, Communication):

1. Compute the relative DCM [BR] at t0
2. Convert [BR] to MRPs
3. Compute the relative angular velocity
4. Express that angular velocity in the body frame
5. Export the results

We are no longer building geometry.
We are measuring tracking error.

This is the first moment where the control law actually becomes meaningful.


In [15]:
def tracking_error(sigma_BN, omega_BN_B, RN, omega_RN_N, t):
    """
    Compute attitude and rate tracking errors between body frame B and reference frame R.

    Inputs
    ------
    sigma_BN : (3,) array_like
        Body attitude relative to inertial, expressed as MRPs (sigma_B/N).
    omega_BN_B : (3,) array_like
        Body angular velocity relative to inertial, expressed in B (B omega_B/N).
    RN : (3,3) array_like or callable
        Passive DCM [RN] mapping N -> R at time t. If callable, must be RN(t)->(3,3).
    omega_RN_N : (3,) array_like or callable
        Reference angular velocity relative to inertial, expressed in N (N omega_R/N) at time t.
        If callable, must be omega_RN_N(t)->(3,).
    t : float
        Time [s] at which to evaluate the reference quantities.

    Outputs
    -------
    sigma_BR : (3,) ndarray
        Attitude error MRPs (sigma_B/R), shadow-set enforced.
    omega_BR_B : (3,) ndarray
        Relative angular velocity B omega_B/R.
    """
    # Evaluate time-varying inputs if needed
    RN_t = RN(t) if callable(RN) else RN
    RN_t = np.asarray(RN_t, dtype=float)
    omega_RN_N_t = omega_RN_N(t) if callable(omega_RN_N) else omega_RN_N

    #print(omega_RN_N_t)

    # --- Attitude error ---
    BN = ak.MRP_to_DCM(sigma_BN)
    BR = BN @ RN_t.T
    sigma_BR = ak.DCM_to_MRP(BR)
    #sigma_BR = ak.MRP_shadow(sigma_BR)

    # --- Angular velocity error ---
    B_omega_RN = BN @ omega_RN_N_t
    omega_BR_B = omega_BN_B - B_omega_RN

    return sigma_BR, omega_BR_B

t0 = 0
sigma_BR, omega_BR = tracking_error(sigma_BN, omega_BN_B, RsN, omega_RsN, t0)
export_txt("task6_part1", sigma_BR)
export_txt("task6_part2", omega_BR)

sigma_BR, omega_BR = tracking_error(sigma_BN, omega_BN_B, RnN, omega_RnN, t0)
export_txt("task6_part3", sigma_BR)
export_txt("task6_part4", omega_BR)

sigma_BR, omega_BR = tracking_error(sigma_BN, omega_BN_B, RcN, omega_RcN, t0)
export_txt("task6_part5", sigma_BR)
export_txt("task6_part6", omega_BR)

[Saved] outputs/task6_part1.txt -> -0.7754 -0.4739 0.0431
[Saved] outputs/task6_part2.txt -> 0.0175 0.0305 -0.0384
[Saved] outputs/task6_part3.txt -> 0.2623 0.5547 0.0394
[Saved] outputs/task6_part4.txt -> 0.0168 0.0309 -0.0389
[Saved] outputs/task6_part5.txt -> 0.0170 -0.3828 0.2076
[Saved] outputs/task6_part6.txt -> 0.0173 0.0307 -0.0384


## Task 7 - Numerical Attitude Simulator

In [16]:
def rk4_step(f, t, x, dt, *args, **kwargs):
    """
    Advance one timestep using classical 4th-order Runge–Kutta (RK4).
    
    Parameters
    ----------
    f : callable
        RHS function with signature f(t, x, *args, **kwargs) -> xdot.
    t : float
        Current time.
    x : np.ndarray
        Current state vector.
    dt : float
        Timestep size.
    *args, **kwargs
        Extra arguments passed through to `f`.
    
    Returns
    -------
    x_next : np.ndarray
        State advanced by one RK4 step of size `dt`.
    """
    k1 = f(t, x, *args, **kwargs)
    k2 = f(t + 0.5*dt, x + 0.5*dt*k1, *args, **kwargs)
    k3 = f(t + 0.5*dt, x + 0.5*dt*k2, *args, **kwargs)
    k4 = f(t + dt, x + dt*k3, *args, **kwargs)
    x_next = x + (dt/6.0)*(k1 + 2*k2 + 2*k3 + k4)
    return x_next

In [17]:
def euler_omega_dot(I_B: np.ndarray, omega_BN_B: np.ndarray, u_B: np.ndarray) -> np.ndarray:
    # I * omega_dot = - omega x (I omega) + u
    H_B = I_B @ omega_BN_B
    omega_cross_H = np.cross(omega_BN_B, H_B)
    omega_dot_B = np.linalg.solve(I_B, (-omega_cross_H + u_B))
    return omega_dot_B

def attitude_xdot(t_s: float, x: np.ndarray, I_B: np.ndarray, u_B: np.ndarray) -> np.ndarray:
    # State: x = [sigma_BN (3), omega_BN_B (3)]
    sigma_BN = x[0:3]
    omega_BN_B = x[3:6]

    sigma_dot = 0.25 * ak.Bmat_MRP(sigma_BN) @ omega_BN_B
    omega_dot = euler_omega_dot(I_B, omega_BN_B, u_B)

    xdot = np.hstack((sigma_dot, omega_dot))
    return xdot

def simulate_rk4(x0: np.ndarray, t0: float, tf: float, dt: float, I_B: np.ndarray, u_func) -> tuple[np.ndarray, np.ndarray]:
    n_steps = int(np.round((tf - t0) / dt))
    t_hist = np.zeros(n_steps + 1)
    x_hist = np.zeros((n_steps + 1, x0.size))

    t_hist[0] = t0
    x_hist[0, :] = x0

    t = t0
    x = x0.copy()

    for k in range(n_steps):
        u_B = np.asarray(u_func(t), dtype=float).reshape(3,)
        x = rk4_step(attitude_xdot, t, x, dt, I_B, u_B)

        # Keep MRPs well-conditioned
        x[0:3] = ak.MRP_shadow(x[0:3])

        t = t + dt
        t_hist[k + 1] = t
        x_hist[k + 1, :] = x

    return t_hist, x_hist

def rotational_kinetic_energy(I_B: np.ndarray, omega_BN_B: np.ndarray) -> float:
    return 0.5 * float(omega_BN_B.T @ I_B @ omega_BN_B)

def angular_momentum_B(I_B: np.ndarray, omega_BN_B: np.ndarray) -> np.ndarray:
    return I_B @ omega_BN_B

def angular_momentum_N(sigma_BN: np.ndarray, H_B: np.ndarray) -> np.ndarray:
    # Passive [BN] maps N -> B, so [NB] = [BN]^T maps B -> N
    BN = ak.MRP_to_DCM(sigma_BN)
    NB = BN.T
    H_N = NB @ H_B
    return H_N

In [21]:
dt = 1  # RK4 with 1 second step is acceptable per brief

x0 = np.hstack((sigma_BN, omega_BN_B))

# Part 1-4: u = 0, propagate to 500 s
u_zero = lambda t: np.array([0.0, 0.0, 0.0])

t_hist, x_hist = simulate_rk4(x0, t0=0.0, tf=500.0, dt=dt, I_B=I, u_func=u_zero)

x_500 = x_hist[-1, :]
sigma_500 = x_500[0:3]
omega_500 = x_500[3:6]

H_B_500 = angular_momentum_B(I, omega_500)
T_500 = rotational_kinetic_energy(I, omega_500)
H_N_500 = angular_momentum_N(sigma_500, H_B_500)

export_txt("task7_part1", H_B_500, precision=12)      # H in B at 500 s
export_txt("task7_part2", T_500, precision=12)        # kinetic energy at 500 s
export_txt("task7_part3", sigma_500, precision=12)    # sigma_BN at 500 s
export_txt("task7_part4", H_N_500, precision=12)      # H in N at 500 s

# Part 5: constant torque, propagate to 100 s
u_const = lambda t: np.array([0.01, -0.01, 0.02])
t_hist2, x_hist2 = simulate_rk4(x0, t0=0.0, tf=100.0, dt=dt, I_B=I, u_func=u_const)
sigma_100 = x_hist2[-1, 0:3]

export_txt("task7_part5", sigma_100, precision=12)


[Saved] outputs/task7_part1.txt -> 0.137897204388 0.132662050721 -0.316387811132
[Saved] outputs/task7_part2.txt -> 0.009384120388
[Saved] outputs/task7_part3.txt -> 0.137659318517 0.560270243755 -0.032172820706
[Saved] outputs/task7_part4.txt -> -0.264126498790 0.252781849536 0.055268751523
[Saved] outputs/task7_part5.txt -> -0.226861107827 -0.641386011151 0.242549803685


# Module 5 - Final Mission Assessment